In [2]:
import os
import hydrofunctions as hf
import pandas as pd
import matplotlib.pyplot as plt
DEBUG = False

# basin = 'merced'
# basin = 'upper san joaquin'
basin = 'tuolumne'
# basin = 'stanislaus'

# variable, pc = 'streamflow', '00060'
variable, pc = 'storage', '00054'

if not os.path.exists(basin):
    os.mkdir(basin)

gages = pd.read_excel('USGS gauges.xlsx', sheet_name='{} - {}'.format(basin, variable), header=None, squeeze=True)

dfs = []
if DEBUG:
    gages = gages[:3]
for i, gage in enumerate(gages):
    gage_number = gage.strip().split(' ')[1]
    print('retrieving {} ({}%)'.format(gage, int((i+1) / len(gages) * 100)))
    herring = hf.NWIS(gage_number, 'dv', '1975-10-01', '2018-09-30', parameterCd=pc)
    try:
        herring.get_data()
    except Exception as err:
        print('Error: ', gage_number, err)
        continue
    df = herring.df()
    s = df[df.columns[0]]
    s.name = gage
    dfs.append(s)
df = pd.concat(dfs, axis=1)
df.index.name = 'Date'
if DEBUG:
    print(df.head())
    df.plot()
    plt.show()
else:
    if variable == 'storage':
        df = df * 1233.5 / 1e6
        path = '{} River/gauges/storage_mcm.csv'.format(basin.title())
    else:
        path = '{} River/gauges/streamflow_cfs.csv'.format(basin.title())
    root_path = os.environ['SIERRA_DATA_PATH']
    outpath = os.path.join(root_path, path)
    df.to_csv(outpath)
print('done!')

retrieving USGS 11275500 HETCH HETCHY RES A HETCH HETCHY CA (25%)
retrieving USGS 11277500 LK ELEANOR NR HETCH HETCHY CA (50%)
retrieving USGS 11277200 CHERRY LK NR HETCH HETCHY CA (75%)
retrieving USGS 11287500 DON PEDRO RES NR LA GRANGE CA (100%)
done!
